<a href="https://colab.research.google.com/github/BriefCud/PRA3024-Big-Data-in-Phys/blob/master/Exoplanet%20and%20Light%20curve%20exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imports
#downloading the necessary imports in order for the prorgam to work
!pip install astroquery
!pip install k2flix
!pip install reproject
%matplotlib inline

import os
import requests
import k2flix

#astroquery
from astroquery.mast import Tesscut
from astroquery.mast import Observations
from astroquery.mast import Catalogs

#astropy
from astropy.io import fits
from astropy.wcs import WCS
from astropy import table
from astropy.coordinates import SkyCoord
from astropy.visualization import simple_norm
import astropy.units as u

import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from reproject import reproject_interp
import pandas as p
from IPython.display import display, HTML, Image
from ipywidgets import interact, interactive, fixed, interact_manual

planeturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/"
dvurl = "https://exo.mast.stsci.edu/api/v0.1/dvdata/tess/"
header={}

In [0]:
def GetInfo(name):

  planet_name = name

  print("Data for ", name)

  url = planeturl + "/identifiers/"

  myparams = {"name":planet_name}

  r = requests.get(url = url, params = myparams, headers = header)
  print(r.headers.get('content-type'))

  planet_names = r.json()
  ticid = planet_names['tessID']
  tce = planet_names['tessTCE']
  planet_names


  url = planeturl + planet_name + "/properties/"

  r = requests.get(url = url, headers = header)

  planet_prop = r.json()

  print("Catalog 1: " + planet_prop[0]['catalog_name'])
  print("Catalog 2: " + planet_prop[1]['catalog_name'])
  planet_prop[0].keys()

  print(planet_name, " Properties")
  print("Stellar Mass %f %s" % (planet_prop[0]['Ms'], planet_prop[0]['Ms_unit'] ) )
  print("Planet Mass %f %s" % (planet_prop[0]['Mp'], planet_prop[0]['Mp_unit'] ) )
  print("Planet Mass Reference: %s" % (planet_prop[0]['Mp_ref']))

  url = dvurl + str(ticid) + '/tces/'
  myparams = {"tce" : tce}

  r = requests.get(url = url, params = myparams, headers = header)
  sectorInfo = r.json()

  sectors = [x[:11] for x in sectorInfo["TCE"] if tce in x]
  sectors

  #using a try except to see if the exoplanet is in the TESS library
  try:
    sectors[0]
  except:
    #if the exoplanet is not in the TESS library print this message
    print("Planet not in TESS library, please choose another planet!")
    return

  url = dvurl + str(ticid) + '/info/'
  myparams = {"tce" : tce, "sector" : sectors[0]}

  r = requests.get(url = url, params = myparams, headers = header)
  tceInfo = r.json()
  tceInfo.keys()
  tceInfo['DV Data Header'].keys()

  print("TIC: %s" % tceInfo['DV Primary Header']['OBJECT'])
  print("Planet Radius: %f" % tceInfo['DV Data Header']['PRADIUS'])

  return(ticid, tce, sectors)

def GetLightCurve(ticid, tce, sectors):

  # Get The data
  url = dvurl + str(ticid) + '/table/'
  myparams = {"tce" : tce, "sector" : sectors[0]}

  r = requests.get(url = url, params = myparams, headers = header)
  tce_data = r.json()

  data = p.DataFrame.from_dict(tce_data['data'])
  data.columns


  #plotting a light curve
  detrend = data['LC_DETREND']
  model = data['MODEL_INIT']
  time = data['TIME']

  plt.figure(figsize=(14,4))
  plt.plot(time,detrend,'.',lw=0.4)
  plt.plot(time,model,'r-',lw=0.6)
  plt.xlabel('TIME (BTJD)')
  plt.ylabel('Relative Flux')



def GetFoldedLightCurve(ticid, tce, sectors):

  #plotting a folded light curve
  url = dvurl + str(ticid) + '/phaseplot/'
  myparams = {"tce" : tce, "sector" : sectors[0]}

  r = requests.get(url = url, params = myparams, headers = header)

  display(HTML(str(r.content.decode('utf-8'))))



def GetGifImage(ticid, tce):

  starName = "TIC" + str(ticid)
  radsearch = 3 / 60
  starData = Catalogs.query_object(starName, radius = radsearch, catalog = "TIC")

  # getting the ra and dec of the star the planet is orbiting 
  ra = starData[0]['ra']
  dec = starData[0]['dec']
  
  target = '{0}, {1}'.format(ra, dec)
  x = y = 11
  # set local file path to current working directory
  path = os.path.abspath(os.path.curdir)

  # create a sky coordinate object
  cutout_coords = SkyCoord(ra, dec, unit="deg")
      
  # download the files and get the list of local paths
  try:
      table = Tesscut.download_cutouts(cutout_coords, size=x, path=path)
  except Exception as e:
      print('Error: Could not download cutouts: {0}'.format(e))
  else:
      #print(table)
      files = table['Local Path']
  
  # grab the first file in the list
  filename = files[0]

  # create the TargetPixelFile instance
  tpf = k2flix.TargetPixelFile(filename)

  # get number of pixels in flux array 
  n_pix = tpf.flux().shape[0]

  # compute field of view in degrees
  res = 21.0 * (u.arcsec/u.pixel)
  area = res * (n_pix * u.pixel)
  d = area.to(u.degree)
  fov = d.value 

  # compute the wcs of the image
  wcs = WCS(tpf.hdulist['APERTURE'].header)

  # print some info
  print('filename', tpf.filename)
  print('Target TPF', target)
  print('Field of View [degrees]', fov)
  print('Number pixels', tpf.flux().shape)

  # create the gif
  gif = tpf.filename.replace('.fits', '.gif')
  if not os.path.exists(gif):
      tpf.save_movie(gif, show_flags=True)
  
  # display the gif
  if not os.path.exists(gif):
      print("No gif found. Cannot display gif of time-series.")

  Image(open(gif, 'rb').read())

  

def main(name):

  planetName = name
  data = []

  data = GetInfo(planetName) 

  #print(data)
  ticid = data[0]
  tce = data[1]
  sectors = data[2]

  GetLightCurve(ticid, tce, sectors)

  #GetFoldedLightCurve(ticid, tce, sectors)

  GetGifImage(ticid, tce)
  
  #---------     NOTES     --------#

  # I have managed to get the gif of the Targeted Pixel file but the gif will not show
  # here even though i have checked it in the Tess workshop files and the coordinates of the planets I used
  # gave a gif image back, but here it doesn't seem to display it

  #the light curve does not seem to appear on my github file but it works in the google colab

  #You might have copy and paste my code into google colab for it work properly


In [0]:
main("HAT-P-30 b")
print(" ")
main("WASP-126 b")